# \[set-up\], imports
See README for more details about installing necessary packages and setting up your environment.

In [ ]:
# This shouldn't really be necessary but it just makes it easier to work wherever
import os
import sys
sys.path.append('/Users/dgood/pint_pal/src') 

In [2]:
import pint_pal as pp
#import pint_pal.par_checker as pc
import pint_pal.lite_utils as lu
import pint_pal.plot_utils as pu
from pint_pal.timingconfiguration import TimingConfiguration
from pint_pal.ftester import run_Ftests
from astropy import log
from pint.fitter import ConvergenceFailure
import pint.fitter
import os
import copy
from astropy.visualization import quantity_support
quantity_support()

# notebook gives interactive plots but not until the kernel is done
%matplotlib notebook
# inline gives non-interactive plots right away
#%matplotlib inline

# Set logging level (PINT uses loguru)
log.setLevel("INFO") # Set desired verbosity of log statements (DEBUG/INFO/WARNING/ERROR)
pint.logging.setup(level="WARNING", usecolors=True)

1

# Pre or Post Noise solution?

In [ ]:
hasnoise = False
if hasnoise == False:
    ext = '_prenoise'
else:
    ext = '_postnoise'

# Load/update timing solution

Load configuration (`.yaml`) file, get TOAs and timing model; if you're running from the root of the git distribution, simply edit the `.yaml` file name, otherwise include relevant paths to the `.yaml` file, and `.par`/`.tim` directories as kwargs (see commented example).

In [ ]:
config = "configs/[psr_name].[nb or wb].yaml"  # fill in actual path
par_directory = None   # default location
tim_directory = None   # default location
tc = TimingConfiguration(config, par_directory=par_directory, tim_directory=tim_directory)

# To combine TOAs, assumption is that cuts have already been applied properly
mo,to = tc.get_model_and_toas(apply_initial_cuts=False,usepickle=False)

# Uncomment this line to manually excise TOAs.
#tc.manual_cuts(to)

# Computing pulse numbers ensures param changes in the model will not break phase connection
to.compute_pulse_numbers(mo)

# Set non-binary epochs to the center of the data span
lu.center_epochs(mo,to)

# Summarize TOAs present
to.print_summary()

In [ ]:
# Define the fitter object and plot pre-fit residuals
fo = tc.construct_fitter(to,mo)
pu.plot_residuals_time(fo, restype='prefit', legend=False)
if mo.is_binary:
    pu.plot_residuals_orb(fo, restype='prefit', legend=False)

In [ ]:
# Set free params based on list in the config file (want to update JUMP handling differently soon)
fo.model.free_params = tc.get_free_params(fo)

# Do the fit
try:
    fo.fit_toas(maxiter=tc.get_niter())
    fo.model.CHI2.value = fo.resids.chi2
except ConvergenceFailure:
    run_Ftest = False
    log.warning('Failed to converge; moving on with best result, but should address before final version.')

In [ ]:
# Plot post-fit residuals, print summary of results, write prenoise solution
pu.plot_residuals_time(fo, restype='postfit', legend=False)
if mo.is_binary:
    pu.plot_residuals_orb(fo, restype='postfit', legend=False)
    
pu.plot_residuals_freq(fo, restype='postfit', legend=False)

    
fo.print_summary()
lu.check_convergence(fo)

lu.write_par(fo,toatype=tc.get_toa_type(),addext=ext,include_date=True)

In [1]:
# Look at the whitened residuals as well
if hasnoise == True:
    pu.plot_residuals_time(fo, restype='postfit', whitened=True, avg=True, mixed_ecorr=True, legend=True)
    if mo.is_binary:
        pu.plot_residuals_orb(fo, restype='postfit', whitened=True, avg=True, mixed_ecorr=True, legend=True)

SyntaxError: 'continue' not properly in loop (1944732518.py, line 7)

# Compare to one other model

You can use this cell to quickly check if/which any parameters have changed by more than 3 sigma (number of sigma is tunable). 

Best use case: compare to a pre-noise model or a previous run. I'm working on a more comprehensive comparison notebook for things like combined vs. individual PTA.

In [ ]:
compmodel = lu.compare_models(fo,
               model_to_compare=tc.get_compare_model(),
               verbosity='min',
               nodmx=True,
               threshold_sigma=3.)

# Check for additional significant parameters (F-test)

This is not likely to be meaningful without noise parameters and it's time consuming -- so we really don't recommend running it until you have some noise parameters. Without that info, it'll be pretty meaningless. 

In [3]:

run_Ftest = False
Ftest_dict = None
if run_Ftest:
    savedLevel = log.getEffectiveLevel()
    try:
        log.setLevel("WARNING")
        Ftest_dict = run_Ftests(fo, 
                                alpha=0.0027, 
                                NITS=tc.get_niter())
    finally:
        log.setLevel(savedLevel)



# \[changelog\] entries

New changelog entries in the `.yaml` file should follow a specific format and are only added for specified reasons (excising TOAs, adding/removing params, changing binary models, etc.). For more detailed instructions, run `lu.new_changelog_entry?` in a new cell. This function can be used to format your entry, which should be added to the bottom of the appropriate `.yaml` file. Note: make sure your git `user.email` is properly configured, since this field is used to add your name to the entry.

In [ ]:
lu.new_changelog_entry?